In [2]:
#import gmaps
#import gmaps.datasets
import numpy as np, matplotlib.pyplot as plt, os, sys, csv
from shapely.geometry import Point, Polygon
import pandas as pd, pickle
%matplotlib inline

# use geojson to import political border info
import gmaps.geojson_geometries
import requests
import numpy.random as random

# need to enable Maps JavaScript API, Directions, Elevation, Geocoding 
APIkey = open('../API_KEY.txt', 'r').readlines()[0]
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

gmaps.configure(api_key=APIkey)

In [5]:
#fig = gmaps.figure(center=center_coords, map_type='ROADMAP', zoom_level=4)

# find usa polygon list from given indices
usa_idx, alaska_idx, mainland_idx = 205, 84, 22
usa_poly_list = countries_geojson['features'][usa_idx]['geometry']['coordinates']

# locate alaska and mainland from list of land masses
alaska_border = [(coord[1], coord[0]) for coord in usa_poly_list[alaska_idx][0]]
mainland_border = [(coord[1], coord[0]) for coord in usa_poly_list[mainland_idx][0]]

#markers = gmaps.marker_layer(mainland_border)
mainland_polygon = gmaps.Polygon(mainland_border,stroke_color='black')
drawing = gmaps.drawing_layer(features=[mainland_polygon],show_controls=False)


In [6]:
## load county coords dataset with from:  https://data.healthcare.gov/dataset/Geocodes-USA-with-Counties/52wv-g36k
## load country voting records from:     https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ/HEIJCQ&version=6.0

## match latitude and longitude to voting record
import csv

# load data from each csv file
df_coords = pd.read_csv('Geocodes_USA_with_Counties.csv')
df_votes = pd.read_csv('countypres_2000-2016.csv')

#add column to show margin of victory:
df_votes['margin'] = df_votes['candidatevotes'] / df_votes['totalvotes']

# refine to 2016 Trump data, remove hawaii and alaska
df_votes = df_votes[df_votes['year'] > 2012]
df_votes = df_votes[df_votes['candidate'] == 'Donald Trump']
df_votes = df_votes[df_votes['state_po'] != 'HI']
df_votes = df_votes[df_votes['state_po'] != 'AK']
print(len(df_votes))

lat_list, long_list = [], []
cnt = 0

for idx, row in df_votes.iterrows():

    # get state and county from voting data
    state, county = row['state_po'], row['county']
    
    # get list of coordinates for major cities in counties
    df_coord_match = df_coords[df_coords['state']==state]
    df_coord_match = df_coord_match[df_coord_match['county']==county]
    
    # take average lat/long of all major cities in count
    lat, long = np.mean(df_coord_match['latitude']), np.mean(df_coord_match['longitude'])
    lat_list.append(lat)
    long_list.append(long)
        
print(len(lat_list), len(long_list))
df_votes['lat'] = lat_list
df_votes['long'] = long_list

#pkl.dump()

3114
3114 3114


In [7]:
#%%time

## write functions for monte carlo cross country
## access elevation from https://nationalmap.gov/epqs/
## https://nationalmap.gov/epqs/pqs.php?x=lat&y=lon&units=Feet&output=json&__ncforminfo=fgwAAffYscBGj_DKa2lIaaTiU5ae6rCJ9xVqysQBA3SVQkLPDv4eLGVmqIj0F4K595Au16fzoWDy09VxzNlbZHBPONkYM4t3Hk7nnMgBCL5LWGHlYiwMDf7-X6rEiHIP8dIjgQ7a-dDf4ck72m2LxQ%3D%3D&__ncforminfo=jZakM3M3ro6f7gTBPacy0976beDgFaH_PyPhQD5GNPnVKUTdBfFsI1SoU6szynyhezFn1VuQ-kF9nd9TjumzunyAsWxQKqgKe2xFI6L6rlzHyfiCH9xpA_78EhU_8EEWl8Ck4G_ajWs20GR5Qsx7pmdF_e0OPKHT

def point_in_poly(point, polygon):
    '''returns whether point is in given polygon'''
    
    pt = Point(*point)
    poly = Polygon(polygon.path)
    return poly.contains(pt)

def get_dist(a, b):
    'get absolute distance between two coords'
    
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
    

def direction_vec(a, b):
    '''get direction unit vector from a to b'''
    
    c = (b[0]-a[0], b[1]-a[1])
    c = c / np.sqrt(c[0]**2 + c[1]**2)
    return c        #(c[0]*step, c[1]*step)


def get_nearest_margin(coords):
    '''returns electoral margin from nearest county to coords'''
    
    df_votes['dist'] = get_dist(coords, (df_votes['lat'], df_votes['long']))
    closest = df_votes.sort_values(by=['dist']).iloc[0]
    return closest['county'], closest['state_po'], closest['margin']


def weighted_move(dest_pointer, current_loc, step=1, dest_bias=0.25, vote_bias=0.25, party='r'):
    '''calculate odds of accepting random moves'''
    
    # generate new location until one is accepted
    max_iter, i = 100, 1
    while i < max_iter:
        
        # get trial move and new location
        trial_vec = (random.uniform(-step, step), random.uniform(-step, step))
        new_loc = (current_loc[0]+trial_vec[0], current_loc[1]+trial_vec[1])
        
        # go to next trial if location not in polygon
        if not point_in_poly(new_loc, mainland_polygon):
            continue
        
        # calculate destination bias based on dot product with dest_vec
        trial_norm = 1/ np.sqrt(trial_vec[0]**2 + trial_vec[1]**2)
        dest_factor = dest_bias*trial_norm*(trial_vec[0]*dest_pointer[0] + trial_vec[1]*dest_pointer[1])
        
        # vote bias with nearest county lookupy
        county, state, margin = get_nearest_margin(new_loc)
        if party=='r': vote_factor = vote_bias*(margin-0.5)
        elif party=='d': vote_factor = vote_bias*(0.5-margin)
        
        prob_accept = 0.5 + dest_factor + vote_factor
        
        # check move is accepted and new location is in polygon
        if random.uniform(0, 1) <  prob_accept:
            new_loc = (current_loc[0]+trial_vec[0], current_loc[1]+trial_vec[1])
            
            return trial_vec, new_loc, i
        
        else: i +=1
    
    
def get_elevation(coords, URL='https://nationalmap.gov/epqs/pqs.php?'):
    '''get elevation with queries (very slow)'''
    
    # sending get request and saving the response as response object 
    PARAMS = {'x':coords[1], 'y':coords[0], 'units':'Feet', 'output':'json'} 
    r = requests.get(url = URL, params = PARAMS)  

    # extracting data in json format 
    data = r.json() 
    elevation = data['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']
    return elevation

#elevation = get_elevation(center_coords)
#print(elevation)


In [8]:
max_step, min_radius = 10000, 0.2  # determines when to end walk
step_size, dest_bias, vote_bias, = 0.1, 0.01, 5.0  # dest_bias must be < 0.5
n_walkers = 1 # number of indepdendent walkers

#start, finish =  (25.7, -80.2), (47.6, -122.3) #miami to seattle    # 
start, finish =  (40.7, -74.00),(34, -118) #new york to la
#start, finish =  (46, -68.4),  (38.9, -77)  # maine to dc

# center map and zoom
#center_coords = (38.6, -90.1) # st louis
center_coords = (42.1, -72.5) # springfield ma
zoom_level = 6

# configure colors of each random walk
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# make fig and add some layers
fig = gmaps.figure(center=center_coords, map_type='TERRAIN', zoom_level=zoom_level)
markers = gmaps.marker_layer([start, finish], label=['start', 'finish'])
fig.add_layer(markers)
fig.add_layer(drawing)

#print(point_in_poly(center_coords, mainland_polygon))

'''# check various bias values
bias_list = [0.1, 0.2, 0.3]
for n, bias in enumerate(bias_list):
    
    color = colors[n]
    current_loc = start
    dist_remaining = get_dist(start, finish)
    feature_list = []
    i = 0
'''

# run multiple walkers for each party
for n in range(n_walkers):
    for party, color in zip(['r', 'd'], ['red', 'blue']):

        current_loc = start
        dist_remaining = get_dist(start, finish)
        feature_list = []
        i = 0

        while i < max_step and dist_remaining > min_radius:

            # find new pointer and location
            dest_pointer = direction_vec(current_loc, finish)
            new_pointer, new_loc, n_iter = weighted_move(dest_pointer, current_loc, step=step_size, dest_bias=0.1, vote_bias=1.0, party=party)

            # mark these on map
            trial_line = gmaps.Line(start=current_loc, end=new_loc, stroke_weight=8.0, stroke_color=color)

            #update location
            current_loc=new_loc
            dist_remaining = get_dist(current_loc, finish)

            # append new features to list
            feature_list.append(trial_line)
            i += 1

        # show where the trip ends
        last_step = gmaps.Marker(current_loc, label=str(i)) #info_box_content=str(i))
        feature_list.append(last_step)
        line_layer = gmaps.drawing_layer(features=feature_list)
        fig.add_layer(line_layer)

fig

Figure(layout=FigureLayout(height='420px'))